In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install tensorflow==2.4.0
!pip install keras==2.4.0
!pip install kymatio
!pip install tqdm
!pip install iterative-stratification
!pip install scikit-multilearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement tensorflow==2.4.0 (from versions: 2.5.0, 2.5.1, 2.5.2, 2.5.3, 2.6.0rc0, 2.6.0rc1, 2.6.0rc2, 2.6.0, 2.6.1, 2.6.2, 2.6.3, 2.6.4, 2.6.5, 2.7.0rc0, 2.7.0rc1, 2.7.0, 2.7.1, 2.7.2, 2.7.3, 2.7.4, 2.8.0rc0, 2.8.0rc1, 2.8.0, 2.8.1, 2.8.2, 2.8.3, 2.8.4, 2.9.0rc0, 2.9.0rc1, 2.9.0rc2, 2.9.0, 2.9.1, 2.9.2, 2.9.3, 2.10.0rc0, 2.10.0rc1, 2.10.0rc2, 2.10.0rc3, 2.10.0, 2.10.1, 2.11.0rc0, 2.11.0rc1, 2.11.0rc2, 2.11.0, 2.11.1, 2.12.0rc0, 2.12.0rc1, 2.12.0)
ERROR: No matching distribution found for tensorflow==2.4.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/sim

In [ ]:
from sklearn.preprocessing import MaxAbsScaler
from sklearn.model_selection import train_test_split, KFold
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers import Adam
import numpy as np
import os
import pickle
import sys
sys.path.append("drive/MyDrive/Scattering_Novo/src")
sys.path.append("drive/MyDrive/DeSpaWN-main")
# Import Data
import scipy.io as sio
from DataHandler import DataHandler
from ModelHandler import ModelHandler
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from skmultilearn.model_selection import iterative_train_test_split
from sklearn.model_selection import KFold
from sklearn.utils.class_weight import compute_class_weight, compute_sample_weight

J=10
Q=2

configs = {
    "N_GRIDS": 5, 
    "SIGNAL_BASE_LENGTH": 12800, 
    "N_CLASS": 26, 
    "USE_NO_LOAD": False, 
    "USE_HAND_AUGMENTATION": False,
    "MARGIN_RATIO": 0.15, 
    "DATASET_PATH": "drive/MyDrive/Scattering_Novo/dataset_original/Synthetic_Full_iHall.hdf5",
    "TRAIN_SIZE": 0.9,
    "FOLDER_PATH": "drive/MyDrive/DeSpaWN-main/extracted_features/without_data_augmentation/Full_Dataset/", 
    "FOLDER_DATA_PATH": "drive/MyDrive/DeSpaWN-main/extracted_features/without_data_augmentation/Full_Dataset/", 
    "FEATURES_FILE_NAME": "features.mat",
    "OUTPUT_CLASSIFICATION_MODELS_PATH": "drive/MyDrive/DeSpaWN-main/classification/without_data_augmentation/",
    "N_EPOCHS_TRAINING": 500,
    "FEAT_NORMALIZATION": False,
    "TRAINING_FLAG": 1,
    "PERCENTUAL": 1.0,
    "INITIAL_EPOCH": 0,
    "TOTAL_MAX_EPOCHS": 5000,
    "SNRdb": None # Nível de ruído em db
}

def freeze(model, task_name='classification'):
    for layer in model.layers:
        if task_name in layer.name:
            layer.trainable = True
        else:
            layer.trainable = False

    for layer in model.layers:
        print(layer.name, layer.trainable)

    return model

def calculating_class_weights(y_true):
    '''
        Source: https://stackoverflow.com/questions/48485870/multi-label-classification-with-class-weights-in-keras
    '''
    from sklearn.utils.class_weight import compute_class_weight
    number_dim = np.shape(y_true)[1]
    weights = np.empty([number_dim, 2])
    for i in range(number_dim):
        weights[i] = compute_class_weight(class_weight='balanced', classes=[0.,1.], y=y_true[:, i])
    return weights


def reduce_dataset(X_all,ydet_all,ytype_all,yclass_all,percentual):
    import numpy as np
    max_index = int(percentual*X_all.shape[0])
    np.random.seed(100)
    index = np.random.randint(max_index,size=(max_index-1))
    X_all = X_all[index]
    ydet_all = ydet_all[index]
    ytype_all = ytype_all[index]
    yclass_all = yclass_all[index]

    return X_all,ydet_all,ytype_all,yclass_all

ngrids = configs["N_GRIDS"]
signalBaseLength = configs["SIGNAL_BASE_LENGTH"]
trainSize = configs["TRAIN_SIZE"]
folderDataPath = configs["FOLDER_DATA_PATH"]
 

folderPath = 'drive/MyDrive/Scattering_Novo/results_second_order/' + 'Hybrid2_P' + str(int(configs['PERCENTUAL']*100)) + '_J' + str(J) + '_Q' + str(Q) + '/'
#folderPath = configs["TESTS_FOLDER"] + 'Hybrid_P' + str(int(configs["PERCENTUAL"]*100)) + '_J' + str(J) + '_Q' + str(Q) + '/'
folderDataPath = configs["FOLDER_DATA_PATH"]

trainSize = configs["TRAIN_SIZE"]

dict_data = pickle.load(open(folderDataPath + "data.p", "rb")) # Load data

modelHandler = ModelHandler(configs)

features_folder_name = configs["FOLDER_PATH"] + "Training_Features" 



In [ ]:
training_flag=configs["TRAINING_FLAG"]

## Load Features

In [ ]:
import scipy.io as sio


### Merging the Training Features

In [ ]:
full_training_set_path = configs["FOLDER_DATA_PATH"] + "combined_features_training_grids2.mat"
full_testing_set_path = configs["FOLDER_DATA_PATH"] + "combined_features_testing_grids2.mat"

training_features_folder_name = configs["FOLDER_DATA_PATH"] + "Training_Features_grids2" 

if not os.path.exists(full_training_set_path):
    #feats= np.zeros([0,14])
    print(full_training_set_path)
    number_of_training_files= 75
    try: 
      for cont in range(number_of_training_files+1): # Number of training files
        path = "features_train_grids2" + str(cont) + ".mat"

        if os.path.isfile(os.path.join(training_features_folder_name,path)):
              csv_path = os.path.join(training_features_folder_name,path)

              print(csv_path)
              #print("Extracting from " + path)
              try:
                imported = sio.loadmat(csv_path)
                imported2 = imported['arr']

                if cont==0:
                    feats = imported2
                else:
                    #imported2 = imported2.reshape([int(imported2.shape[0]/14),14]) # The shape of a single file
                    feats = np.append(feats,  imported2, axis=0 )
              except:
                print("Fail merging file " + path)
        else:
              print("File is not from training dataset")
    except:
      print("Training Directory does not exist")

    sio.savemat(full_training_set_path, {'arr': feats})
    print("Combined and saved!")

    print(feats.shape)

    del feats, imported2


In [ ]:
print("Training data exists? ", os.path.exists(full_training_set_path))
print("Testing data exists? ", os.path.exists(full_testing_set_path))


Training data exists?  True
Testing data exists?  True


In [ ]:
X_train = sio.loadmat(full_training_set_path)
X_train = X_train['arr']
y_train = sio.loadmat(configs["FOLDER_DATA_PATH"] + "yclass_train_.mat")
y_train = y_train['arr']


In [ ]:
print(X_train.shape)
print(y_train.shape)

(7575, 5, 10)
(7575, 5, 26)


In [ ]:
import numpy as np

A = np.zeros([20,5,10])

B = np.array([])

for k in range(A.shape[0]):
  if k==0:
    B = A[k].flatten()
    B = B.reshape([1,B.shape[0]])
  else:
    b = A[k].flatten()
    b = b.reshape([1,b.shape[0]])
    B = np.append(B, b, axis=0)

print(A.shape)
print(B.shape)

(20, 5, 10)
(20, 50)


In [ ]:
y_train[10]

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

### Merging the testing features

In [ ]:
number_of_testing_files= 8
#features_folder_name = configs["FOLDER_DATA_PATH"] + "Testing_Features" 
testing_features_folder_name = configs["FOLDER_DATA_PATH"] + "Testing_Features_grids2" 

full_testing_set_path = configs["FOLDER_DATA_PATH"] + "combined_features_testing_grids2.mat"

if not os.path.exists(full_testing_set_path):
    try: 
      for cont in range(number_of_testing_files+1): # Number of training files
        path = "features_test_grids2" + str(cont) + ".mat"

        if os.path.isfile(os.path.join(testing_features_folder_name,path)):
              csv_path = os.path.join(testing_features_folder_name,path)
              #print("Extracting from " + path)
              try:
                imported = sio.loadmat(csv_path)
                imported2 = imported['arr']
                #imported2 = imported2.reshape([imported2.shape[1],])

                if cont==0:
                    feats = imported2
                else:
                    #imported2 = imported2.reshape([int(imported2.shape[0]/14),14]) # The shape of a single file
                    feats = np.append(feats,  imported2, axis=0 )
              except:
                print("Fail merging file " + path)
        else:
              print("File is not from testing dataset")
    except:
      print("Training Directory does not exist")

    sio.savemat(configs["FOLDER_PATH"] + "combined_features_testing_grids2" + ".mat", {'arr': feats})
    print("Combined and saved!")
    del feats, imported2
else:
    print("Testing data already merged.")

Testing data already merged.


In [ ]:
os.path.exists(full_training_set_path)

True

In [ ]:
print(full_testing_set_path)

X_test  = sio.loadmat(full_testing_set_path)
X_test = X_test['arr']
y_test  = sio.loadmat(configs["FOLDER_DATA_PATH"] + "yclass_test_.mat")
y_test = y_test['arr']


drive/MyDrive/DeSpaWN-main/extracted_features/without_data_augmentation/Full_Dataset/combined_features_testing_grids2.mat


In [ ]:
print("Shape of Classification Testing Features: ", X_test.shape)
print("Shape of Classification Testing Labels: ", y_test.shape)


Shape of Classification Testing Features:  (841, 5, 10)
Shape of Classification Testing Labels:  (841, 5, 26)


## Features Normalization

In [ ]:
if configs["FEAT_NORMALIZATION"]:
  from sklearn.preprocessing import MultiLabelBinarizer
  from sklearn.preprocessing import MaxAbsScaler 
  for grid in range(5): 
    if grid==0:
      transformer = MaxAbsScaler().fit(X_test[:,grid,:])
      x_test_norm = transformer.transform(X_test[:,grid,:])
      transformer = MaxAbsScaler().fit(X_train[:,grid,:])
      X_train_norm = transformer.transform(X_train[:,grid,:])
    else:
      transformer = MaxAbsScaler().fit(X_test[:,grid,:])
      x_test_norm = np.append(x_test_norm, transformer.transform(X_test[:,grid,:]), axis=0 )
      transformer = MaxAbsScaler().fit(X_train[:,grid,:])
      X_train_norm = np.append( X_train_norm, transformer.transform(X_train[:,grid,:]), axis=0)

  x_test_norm = x_test_norm.reshape([-1,5,10])
  X_train_norm = X_train_norm.reshape([-1,5,10])

  print(x_test_norm.shape)
  print(X_train_norm.shape)

  X_test = x_test_norm
  X_train = X_train_norm

  del x_test_norm, X_train_norm

## Training

In [ ]:
def buildOutputModel_LW(input_class_shape):
        
        import tensorflow as tf
        from keras import backend as K
        from keras.layers import Input, Conv1D, LeakyReLU, MaxPooling1D, Dropout, Dense, Reshape, Flatten, Softmax, GlobalAveragePooling1D, Lambda
        from keras.models import Model, load_model
        from keras.utils.vis_utils import plot_model
        #from kymatio.keras import Scattering1D

        
        input_class = Input(shape=input_class_shape)


        classification_output = Dense(300, name='classification_dense_0')(input_class)
        classification_output = LeakyReLU(alpha = 0.1, name='classification_leaky_0')(classification_output)
        classification_output = Dropout(0.25, name='classification_dropout')(classification_output)
        classification_output = Dense(300, name='classification_dense_1')(classification_output)
        classification_output = LeakyReLU(alpha=0.1, name='classification_leaky_1')(classification_output)
        classification_output = Dense((configs["N_CLASS"]) * configs["N_GRIDS"], activation = 'sigmoid', name='classification_dense_2')(classification_output)
        classification_output = Reshape((configs["N_GRIDS"], (configs["N_CLASS"])), name = "classification")(classification_output)


       
        model = Model(inputs = [input_class], outputs=[classification_output])

        return model 

In [ ]:

fold = 0
mskf = MultilabelStratifiedKFold(n_splits=10, shuffle=True, random_state=42)
strat_classes = np.max(y_train, axis=1)
print(strat_classes.shape)

yclass_all = y_train

for train_index, validation_index in mskf.split(X_train, strat_classes):
    fold += 1

    if fold != 10:
        continue

    print(f"---------- FOLD {fold} -------------")

    print("Size of extracted classification features: ", X_train.shape)

    y_train, y_validation = {}, {}
    
    y_train["classification"] = yclass_all[train_index]
    y_validation["classification"] = yclass_all[validation_index]

   
    x_train_class = X_train[train_index]
    x_validation_class = X_train[validation_index]

    # Flattening train

    print("Shape do treino antes do flatten: ", x_train_class.shape)
    print("Shape da validação antes do flatten: ", x_validation_class.shape)

    for k in range(x_train_class.shape[0]):
      if k==0:
        B = x_train_class[k].flatten()
        B = B.reshape([1,B.shape[0]])
      else:
        b = x_train_class[k].flatten()
        b = b.reshape([1,b.shape[0]])
        B = np.append(B, b, axis=0)
    x_train_class = B
    del B

    # Flattening validation

    for k in range(x_validation_class.shape[0]):
      if k==0:
        B = x_validation_class[k].flatten()
        B = B.reshape([1,B.shape[0]])
      else:
        b = x_validation_class[k].flatten()
        b = b.reshape([1,b.shape[0]])
        B = np.append(B, b, axis=0)
    x_validation_class = B
    del B


    print("Flattened validation shape: ", x_validation_class.shape)
    print("Flattened training shape: ", x_train_class.shape)

    yclass_weights = calculating_class_weights(np.max(y_train["classification"], axis=1))

    print(yclass_weights)
    
    folderPath = configs["OUTPUT_CLASSIFICATION_MODELS_PATH"] + "FC_OUTPUT_CLASS/"
    
    try:
        os.mkdir(folderPath)
    except:
        pass

    folderPath = configs["OUTPUT_CLASSIFICATION_MODELS_PATH"] + "FC_OUTPUT_CLASS/" + str(fold) + "/"

    try:
        os.mkdir(folderPath)
    except:
        pass
    try:
        os.mkdir('./' + configs["OUTPUT_CLASSIFICATION_MODELS_PATH"] + "FC_OUTPUT_CLASS/" + 'logs')
    except:
        pass

    np.save(folderPath + "train_index.npy", train_index)
    np.save(folderPath + "validation_index.npy", validation_index)
    

    #tensorboard_callback = TensorBoard(log_dir='./' + configs["FOLDER_PATH"] + '/logs')
    tensorboard_callback = TensorBoard(log_dir='./' + configs["OUTPUT_CLASSIFICATION_MODELS_PATH"] + "FC_OUTPUT_CLASS/" + 'logs')

    # early_stopping_callback = EarlyStopping(monitor='val_loss', patience=50, verbose=True, restore_best_weights=True)

    if configs["INITIAL_EPOCH"] > 0:
        model = ModelHandler.loadModel(folderPath + 'model_{0}.h5'.format(configs["INITIAL_EPOCH"]))
    else:
        model = buildOutputModel_LW(input_class_shape=x_train_class.shape[1]) # Perceba que aqui a função de construção foi alterada
 
    model.summary()
 
    fileEpoch = configs["INITIAL_EPOCH"]
    while fileEpoch < configs["TOTAL_MAX_EPOCHS"]:
        fileEpoch += configs["N_EPOCHS_TRAINING"]      

        if not os.path.isfile(folderPath + 'model_without_detection.h5'):
            #for subtask in ['type', 'classification']:
            #for subtask in ['classification']:
                #print(f"FOLD {fold}: Training {subtask}")
                subtask = 'classification'   
                print(f"FOLD {fold}: Training {subtask}")
                #freeze(model, task_name=subtask)
                model.compile(optimizer = Adam(), \
                            #   loss = [ModelHandler.sumSquaredError, "categorical_crossentropy", ModelHandler.get_bce_weighted_loss(yclass_weights)], \
                            #   loss = [ModelHandler.sumSquaredError, "categorical_crossentropy", "binary_crossentropy"], \
                            #  loss = ["categorical_crossentropy", "binary_crossentropy"], \
                              loss = ["binary_crossentropy"], \
                            #  metrics=[['categorical_accuracy'], ['binary_accuracy']])
                              metrics=[['binary_accuracy']])
                
                #early_stopping_callback = EarlyStopping(monitor=f"val_{subtask}_loss", patience=50, verbose=True, restore_best_weights=True)
                early_stopping_callback = EarlyStopping(monitor=f"val_loss", patience=50, verbose=True, restore_best_weights=True)

                #hist_opt = model.fit(x=x_train, y=[y_train["type"], y_train["classification"]], \
                #                    validation_data=(x_validation, [y_validation["type"], y_validation["classification"]]), \
                #                    epochs=configs["N_EPOCHS_TRAINING"], verbose=2, callbacks=[early_stopping_callback, tensorboard_callback], batch_size=32)
                hist_opt = model.fit(x=[x_train_class], y=[y_train["classification"]], \
                                    validation_data=([x_validation_class], [y_validation["classification"]]), \
                                    epochs=configs["N_EPOCHS_TRAINING"], verbose=2, callbacks=[early_stopping_callback, tensorboard_callback], batch_size=32)
                
                   
                model.save(folderPath + 'model_without_detection.h5')
  
    del model, y_train, x_train_class, x_validation_class

(7575, 26)
---------- FOLD 10 -------------
Size of extracted classification features:  (7575, 5, 10)
Shape do treino antes do flatten:  (6801, 5, 10)
Shape da validação antes do flatten:  (774, 5, 10)
Flattened validation shape:  (774, 50)
Flattened training shape:  (6801, 50)
[[  0.53907736   6.89756592]
 [  0.54802579   5.70553691]
 [  0.50199291 125.94444444]
 [  0.58468019   3.45228426]
 [  0.60766619   2.8219917 ]
 [  0.50191882 130.78846154]
 [  0.5292607    9.04388298]
 [  0.62658928   2.47489083]
 [  0.56836035   4.15709046]
 [  0.50184475 136.02      ]
 [  0.50966727  26.36046512]
 [  0.53907736   6.89756592]
 [  0.64981846   2.16868622]
 [  0.63265116   2.38464236]
 [  0.50191882 130.78846154]
 [  0.56703352   4.22947761]
 [  0.61050269   2.7623883 ]
 [  0.52712758   9.71571429]
 [  0.52917834   9.068     ]
 [  0.52194935  11.88986014]
 [  0.52704588   9.74355301]
 [  0.52819199   9.3677686 ]
 [  0.53149422   8.43796526]
 [  0.53796868   7.084375  ]
 [  0.53249295   8.193975